In [2]:
from PyQt5 import QtWidgets
from PyQt5 import sip
from GUI8 import Ui_MainWindow
from PyQt5.QtWidgets import QFileDialog,QMessageBox
import sys
from PyQt5 import QtCore, QtGui, QtWidgets
from PyQt5.QtWidgets import QDialog, QApplication, QPushButton, QVBoxLayout
from PyQt5 import QtCore, QtWidgets, QtMultimedia
from PyQt5 import QtWidgets
import sys
import numpy as np
import random
import math
import cv2
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
%matplotlib inline  
import time 
import os
from numpy import pi, exp, sqrt
import scipy.stats as st
from scipy import signal
from PIL import Image, ImageOps
import logging
import pyqtgraph as pg
random.seed(1234)

logging.basicConfig(filename='logging.log', level=logging.INFO,
                    format='%(levelname)s:%(message)s')

class ApplicationWindow(QtWidgets.QMainWindow):
    flag=0
    path1 = "" 
    path2 = ""
    def __init__(self):
        super(ApplicationWindow, self).__init__()
        self.ui = Ui_MainWindow()
        self.ui.setupUi(self)
        
        self.ui.img1.triggered.connect(self.loaddata)
        self.ui.img2.triggered.connect(self.loaddata2)
        self.ui.selectimg.currentTextChanged.connect(self.combobox_text)
        self.ui.noise.currentTextChanged.connect(self.noise)
        self.ui.filter.currentTextChanged.connect(self.noise)
        self.ui.edge.currentTextChanged.connect(self.edge)
        self.ui.histogram.clicked.connect(self.histogram)
        self.ui.curve.clicked.connect(self.distribution_curve)
        self.ui.normalize.clicked.connect(self.norm)
        self.ui.local_butt.clicked.connect(self.local_thr)
        self.ui.global_butt.clicked.connect(self.global_thr)
        self.ui.equalize.clicked.connect(self.equalize)
        self.ui.pushButton.clicked.connect(self.freq_filter)
        self.ui.rgb.clicked.connect(self.RGB_histo)
        self.ui.hybrid.clicked.connect(self.hybridImage)
        
        
        self.histo_data =  self.ui.histogram_view.plotItem.plot()
        self.curve_data =  self.ui.curve_view.plotItem.plot()
        self.histo_r =  self.ui.histo_r.plotItem.plot()
        self.histo_g =  self.ui.histo_g.plotItem.plot()
        self.histo_b =  self.ui.histo_b.plotItem.plot()
    
        
    def loaddata(self):
        filename = QFileDialog.getOpenFileName(self)
        if filename[0]:
            self.path1 = filename[0]
            #self.get_image(self.path1)
    def loaddata2(self):
        filename = QFileDialog.getOpenFileName(self)
        if filename[0]:
            self.path2 = filename[0]
            #self.get_image(self.path1)
            

    def combobox_text(self):
        self.img_name = self.ui.selectimg.currentText()
        self.process(self.img_name)
        self.noise()
    
    def rgb2graynew(self,image):
        grayImage = np.zeros(image.shape)
        R = np.array(image[:, :, 0])
        G = np.array(image[:, :, 1])
        B = np.array(image[:, :, 2])

        R = (R *.299)
        G = (G *.587)
        B = (B *.114)

        Avg = (R+G+B)
        grayImage = image

        for i in range(3):
           grayImage[:,:,i] = Avg

        return grayImage
    
    def process(self,image):
        if(image == "Image 1"):
            path = self.path1
        elif(image == "Image 2"):
            path = self.path2
        elif(image == "Select Image"):
            return 0
        self.image_color, self.image_gray = self.read_image(path)
        #self.img1_size = self.img1.shape
        color_img = np.copy(self.image_color)
        color2Gray = self.rgb2graynew(color_img)
        self.ui.colorimg.show()
        self.ui.colorimg.setImage(np.rot90(self.image_color, 1))
        self.ui.grayimg.show()
        self.ui.grayimg.setImage(np.rot90(color2Gray,1))
            
    def read_image(self,path):
        self.image_color = Image.open(path)
        self.image_color = np.array(self.image_color)
        self.image_gray = np.transpose(cv2.imread(path, 0))
        return self.image_color, self.image_gray
    
    def noise(self):
        self.noise_name = self.ui.noise.currentText()
        self.img_name = self.ui.selectimg.currentText()
        if(self.img_name == "Image 1"):
            path = self.path1
        elif(self.img_name == "Image 2"):
            path = self.path2  
        self.image_color, self.image_gray = self.read_image(path)
        noisy_out = self.image_gray
        if(self.noise_name == "Uniform Noise"):
            noisy_img = self.image_gray +  np.random.rand(512,512)
            noisy_out = np.clip(noisy_img, 0, 255)
            self.ui.noise_view.show()
            self.ui.noise_view.setImage(noisy_out)
        elif(self.noise_name == "Gaussian Noise"):
            noisy_img = self.image_gray + np.random.normal(0, 10, self.image_gray.shape)
            noisy_out = np.clip(noisy_img, 0, 255)
            self.ui.noise_view.show()
            self.ui.noise_view.setImage(noisy_out)
        elif(self.noise_name == "Salt & Pepper Noise"):
            row , col =  self.image_gray.shape
            number_of_pixels = random.randint(300, 1000)
            noisy_image = np.copy(self.image_gray)
            for i in range(number_of_pixels):
                y_coord=random.randint(0, row - 1)
                x_coord=random.randint(0, col - 1)
                noisy_image[y_coord][x_coord] = 255
            number_of_pixels = random.randint(300 , 1000)
            for i in range(number_of_pixels):
                y_coord=random.randint(0, row - 1)
                x_coord=random.randint(0, col - 1)
                noisy_image[y_coord][x_coord] = 0
            noisy_out = noisy_image
            self.ui.noise_view.show()
            self.ui.noise_view.setImage(noisy_out)
            
        elif(self.noise_name == "Adding Noise"):
            self.ui.noise_view.show()
            self.ui.noise_view.setImage(self.image_gray)
        self.filter(noisy_out,self.image_gray)
        
    
    def padding(self,f,image):
        n, m = image.shape
        pad = int((f-1)/2)
        padded_image = np.zeros((image.shape[0] + 2*pad, image.shape[1] + 2*pad ))
        padded_image[pad:padded_image.shape[0]-pad,pad:padded_image.shape[1]-pad ] = image

        return padded_image , pad 
    
    def conv(self,image,kernel):
        f = kernel.shape[0]
        kernel = np.flipud(np.fliplr(kernel))
        n, m = image.shape
        padded_image, pad = self.padding(f,image)
        out_img = np.zeros((n-f+(2*pad)+1, n-f+(2*pad)+1))
        for j in range(n-f+1):
                for i in range(n-f+1):
                    out_img[i,j] = (kernel*padded_image[i:i+f, j:j+f]).sum()
        return out_img
    
    def gkern(self,s,k):
        #  generate a (2k+1)x(2k+1) gaussian kernel with mean=0 and sigma = s
        probs = [exp(-z*z/(2*s*s))/sqrt(2*pi*s*s) for z in range(-k,k+1)] 
        kernel = np.outer(probs, probs)
        return kernel
    def avg_filter(self,f):
        kernel = np.ones((f,f)) / (f**2)
        return kernel
    
    def median_filter(self,image,f):
        n, m = image.shape
        index = int(((f**2) + 1) / 2)
        padded_image, pad = self.padding(f,image)
        out_img = np.zeros((n-f+(2*pad)+1, n-f+(2*pad)+1))
        for j in range(n-f+1):
            for i in range(n-f+1):
                out_img[i,j] = np.sort(padded_image[i:i+f, j:j+f].reshape(-1))[index]
        return out_img

    def filter(self,image,origenal_img):
        self.filter_name = self.ui.filter.currentText()
        
        if(self.filter_name == "Average Filter"):
            filtered_img = self.conv(image,self.avg_filter(21))
        elif(self.filter_name == "Gaussian Filter"):
            filtered_img = self.conv(image,self.gkern(1,5))
        elif(self.filter_name == "Median Filter"):
            filtered_img = self.median_filter(image,13) 
        elif(self.filter_name == "Apply Low Pass Filter"):
            filtered_img = origenal_img
        self.ui.filter_view.show()
        self.ui.filter_view.setImage(filtered_img)
            
    
    def edge(self):
        self.img_name = self.ui.selectimg.currentText()
        self.edge_name = self.ui.edge.currentText()
        if(self.img_name == "Image 1"):
            path = self.path1
        elif(self.img_name == "Image 2"):
            path = self.path2
        self.image_color, self.image_gray = self.read_image(path)
        image = self.image_gray
        if(self.edge_name == "Sobel Mask"):
            sobel_x_kernal = np.array([[-1, -2, -1], [0, 0, 0], [1, 2, 1]])
            sobel_y_kernal = np.array([[-1, 0, 1], [-2, 0, 2], [-1, 0, 1]])
            sobel_x = self.conv(image,sobel_x_kernal)
            sobel_y = self.conv(image,sobel_y_kernal)
            sobel_mag = np.sqrt((sobel_x**2) + (sobel_y**2))
            sobel_angle = np.arctan2(sobel_x, sobel_y)
            sobel_angle = np.rad2deg(sobel_angle)
            sobel_angle += 180
            self.ui.edge_view.show()
            self.ui.edge_view.setImage(sobel_mag)
            
        elif(self.edge_name == "Roberts Mask"):
            Robert_x_kernal = np.array([[1, 0], [0, -1]])
            Robert_y_kernal = np.array([[0, 1], [-1, 0]])
            Robert_x = self.conv(image,Robert_x_kernal)
            Robert_y = self.conv(image,Robert_y_kernal)
            Robert_mag = np.sqrt((Robert_x**2) + (Robert_y**2))
            Robert_angle = np.arctan2(Robert_x, Robert_y)
            Robert_angle = np.rad2deg(Robert_angle)
            Robert_angle += 180
            self.ui.edge_view.show()
            self.ui.edge_view.setImage(Robert_mag)
            
        elif(self.edge_name == "Prewitt Mask"):
            perwit_x_kernal = np.array([[-1, -1, -1], [0, 0, 0], [1, 1, 1]])
            perwit_y_kernal = np.array([[-1, 0, 1], [-1, 0, 1], [-1, 0, 1]])
            perwit_x = self.conv(image,perwit_x_kernal)
            perwit_y = self.conv(image,perwit_y_kernal)
            perwit_mag = np.sqrt((perwit_x**2) + (perwit_y**2))
            perwit_angle = np.arctan2(perwit_x, perwit_y)
            perwit_angle = np.rad2deg(perwit_angle)
            perwit_angle += 180
            self.ui.edge_view.show()
            self.ui.edge_view.setImage(perwit_mag)
            
        elif(self.edge_name == "Canny Mask"):
            sobel_x_kernal = np.array([[-1, -2, -1], [0, 0, 0], [1, 2, 1]])
            sobel_y_kernal = np.array([[-1, 0, 1], [-2, 0, 2], [-1, 0, 1]])
            Gauiss_img = self.conv(image,self.gkern(1,5))
            sobel_img_x = self.conv(Gauiss_img,sobel_x_kernal)
            sobel_img_y = self.conv(Gauiss_img,sobel_y_kernal)
            sobel_mag = np.sqrt((sobel_img_x**2) + (sobel_img_y**2))
            gradient_direction = np.arctan2(sobel_img_x, sobel_img_y)
            gradient_direction = np.rad2deg(gradient_direction)
            gradient_direction += 180
            new_image1 = self.non_max_suppression(sobel_mag, gradient_direction)
            weak = 50
            new_image2 = self.threshold(new_image1, 10, 70, weak=weak)
            new_image = self.hysteresis(new_image2, weak)
            self.ui.edge_view.show()
            self.ui.edge_view.setImage(new_image)
            
        elif(self.edge_name == "Edge Detection"):
            self.ui.edge_view.show()
            self.ui.edge_view.setImage(image)
    
    
    ## cany edge detection 

    def non_max_suppression(self,gradient_magnitude, gradient_direction):
        image_row, image_col = gradient_magnitude.shape

        output = np.zeros(gradient_magnitude.shape)

        PI = 180

        for row in range(1, image_row - 1):
            for col in range(1, image_col - 1):
                direction = gradient_direction[row, col]

                if (0 <= direction < PI / 8) or (15 * PI / 8 <= direction <= 2 * PI):
                    before_pixel = gradient_magnitude[row, col - 1]
                    after_pixel = gradient_magnitude[row, col + 1]

                elif (PI / 8 <= direction < 3 * PI / 8) or (9 * PI / 8 <= direction < 11 * PI / 8):
                    before_pixel = gradient_magnitude[row + 1, col - 1]
                    after_pixel = gradient_magnitude[row - 1, col + 1]

                elif (3 * PI / 8 <= direction < 5 * PI / 8) or (11 * PI / 8 <= direction < 13 * PI / 8):
                    before_pixel = gradient_magnitude[row - 1, col]
                    after_pixel = gradient_magnitude[row + 1, col]

                else:
                    before_pixel = gradient_magnitude[row - 1, col - 1]
                    after_pixel = gradient_magnitude[row + 1, col + 1]

                if gradient_magnitude[row, col] >= before_pixel and gradient_magnitude[row, col] >= after_pixel:
                    output[row, col] = gradient_magnitude[row, col]


        return output

    def threshold(self,image, low, high, weak):
        output = np.zeros(image.shape)

        strong = 255

        strong_row, strong_col = np.where(image >= high)
        weak_row, weak_col = np.where((image <= high) & (image >= low))

        output[strong_row, strong_col] = strong
        output[weak_row, weak_col] = weak

        return output

    def hysteresis(self,image, weak):
        image_row, image_col = image.shape

        top_to_bottom = image.copy()

        for row in range(1, image_row):
            for col in range(1, image_col):
                if top_to_bottom[row, col] == weak:
                    if top_to_bottom[row, col + 1] == 255 or top_to_bottom[row, col - 1] == 255 or top_to_bottom[row - 1, col] == 255 or top_to_bottom[
                        row + 1, col] == 255 or top_to_bottom[
                        row - 1, col - 1] == 255 or top_to_bottom[row + 1, col - 1] == 255 or top_to_bottom[row - 1, col + 1] == 255 or top_to_bottom[
                        row + 1, col + 1] == 255:
                        top_to_bottom[row, col] = 255
                    else:
                        top_to_bottom[row, col] = 0

        bottom_to_top = image.copy()

        for row in range(image_row - 1, 0, -1):
            for col in range(image_col - 1, 0, -1):
                if bottom_to_top[row, col] == weak:
                    if bottom_to_top[row, col + 1] == 255 or bottom_to_top[row, col - 1] == 255 or bottom_to_top[row - 1, col] == 255 or bottom_to_top[
                        row + 1, col] == 255 or bottom_to_top[
                        row - 1, col - 1] == 255 or bottom_to_top[row + 1, col - 1] == 255 or bottom_to_top[row - 1, col + 1] == 255 or bottom_to_top[
                        row + 1, col + 1] == 255:
                        bottom_to_top[row, col] = 255
                    else:
                        bottom_to_top[row, col] = 0

        right_to_left = image.copy()

        for row in range(1, image_row):
            for col in range(image_col - 1, 0, -1):
                if right_to_left[row, col] == weak:
                    if right_to_left[row, col + 1] == 255 or right_to_left[row, col - 1] == 255 or right_to_left[row - 1, col] == 255 or right_to_left[
                        row + 1, col] == 255 or right_to_left[
                        row - 1, col - 1] == 255 or right_to_left[row + 1, col - 1] == 255 or right_to_left[row - 1, col + 1] == 255 or right_to_left[
                        row + 1, col + 1] == 255:
                        right_to_left[row, col] = 255
                    else:
                        right_to_left[row, col] = 0

        left_to_right = image.copy()

        for row in range(image_row - 1, 0, -1):
            for col in range(1, image_col):
                if left_to_right[row, col] == weak:
                    if left_to_right[row, col + 1] == 255 or left_to_right[row, col - 1] == 255 or left_to_right[row - 1, col] == 255 or left_to_right[
                        row + 1, col] == 255 or left_to_right[
                        row - 1, col - 1] == 255 or left_to_right[row + 1, col - 1] == 255 or left_to_right[row - 1, col + 1] == 255 or left_to_right[
                        row + 1, col + 1] == 255:
                        left_to_right[row, col] = 255
                    else:
                        left_to_right[row, col] = 0

        final_image = top_to_bottom + bottom_to_top + right_to_left + left_to_right

        final_image[final_image > 255] = 255

        return final_image
    
    def histogram(self):
        self.img_name = self.ui.selectimg.currentText()
        if(self.img_name == "Image 1"):
            path = self.path1
        elif(self.img_name == "Image 2"):
            path = self.path2
        pen = pg.mkPen(color=(0, 200, 0))
        self.image_color, self.image_gray = self.read_image(path)
        histogram, bin_counts = np.histogram(self.image_gray, bins=256)
        self.histo_data.setData(histogram,pen=pen)
        
    def distribution_curve(self):
        self.img_name = self.ui.selectimg.currentText()
        if(self.img_name == "Image 1"):
            path = self.path1
        elif(self.img_name == "Image 2"):
            path = self.path2
        pen = pg.mkPen(color=(0, 200, 0))
        self.image_color, self.image_gray = self.read_image(path)
        histogram, bin_counts = np.histogram(self.image_gray, bins=256)
        mu = 10
        sigma = 20
        y = ((1 / (np.sqrt(2 * np.pi) * sigma)) *
             np.exp(-0.5 * (1 / sigma * (bin_counts - mu))**2))
        y = y.cumsum()
        y /= y[-1]
        self.curve_data.setData(y,pen=pen)  
        
    def equalize(self):
        self.img_name = self.ui.selectimg.currentText()
        if(self.img_name == "Image 1"):
            path = self.path1
        elif(self.img_name == "Image 2"):
            path = self.path2
        self.image_color, self.image = self.read_image(path)
        
        #STEP 1: Normalized cumulative histogram
        histogram_array = np.bincount(self.image.flatten(), minlength=256)
        #plt.plot(histogram_array)  # <- or here
        #plt.show()
        num_pixels = np.sum(histogram_array)
        histogram_array = histogram_array/num_pixels
        chistogram_array = np.cumsum(histogram_array)
        #STEP 2: Pixel mapping lookup table
        transform_map = np.floor(255 * chistogram_array).astype(np.uint8)
        #STEP 3: Transformation
        img_list = list(self.image.flatten())
        eq_img_list = [transform_map[p] for p in img_list]
        eq_img_array = np.reshape(np.asarray(eq_img_list), self.image.shape)
        
        self.ui.original_img.show()
        self.ui.original_img.setImage(self.image)
        self.ui.equalize_view.show()
        self.ui.equalize_view.setImage(eq_img_array)
        
    
    def norm(self):
        self.img_name = self.ui.selectimg.currentText()
        if(self.img_name == "Image 1"):
            path = self.path1
        elif(self.img_name == "Image 2"):
            path = self.path2
        self.image_color, self.image = self.read_image(path)
        pixels = np.asarray(self.image)
        pixels = pixels.astype('float32')
        pixels /= 255.0
        self.ui.normalize_view.show()
        self.ui.normalize_view.setImage(pixels)
        
    def global_thr(self):
        self.img_name = self.ui.selectimg.currentText()
        if(self.img_name == "Image 1"):
            path = self.path1
        elif(self.img_name == "Image 2"):
            path = self.path2
        self.image_color, self.image = self.read_image(path)
        threshold = 125
        thresImage = np.where(self.image > threshold, 255, 0)
        self.ui.global_view.show()
        self.ui.global_view.setImage(thresImage)
    
    def local_thr(self):
        self.img_name = self.ui.selectimg.currentText()
        if(self.img_name == "Image 1"):
            path = self.path1
        elif(self.img_name == "Image 2"):
            path = self.path2
        self.image_color, self.image = self.read_image(path)
        avgfilter = np.ones((5,5)) / (25)
        pixels = self.conv(self.image, avgfilter)
        threshold = 125
        thresImage = np.where(pixels > threshold, 255, 0)
        self.ui.local_view.show()
        self.ui.local_view.setImage(thresImage)
    
    
    def fourier (self,image):
        fimage = np.fft.fft2(image)
        shiftedimage = np.fft.fftshift(fimage)
        return (shiftedimage)

    def fourier_inverse (self,image):
        inverse = np.fft.ifftshift(image)
        img = np.fft.ifft2(inverse)
        return (img)

    def freq_filter(self):
        self.img_name = self.ui.selectimg.currentText()
        if(self.img_name == "Image 1"):
            path = self.path1
        elif(self.img_name == "Image 2"):
            path = self.path2
        self.image_color, self.image = self.read_image(path)
        img_f = self.fourier(self.image)
        lp_filter = np.zeros(self.image.shape)
        lp_filter[20:41,20:41] = self.avg_filter(21)
        lp_filter = self.fourier(lp_filter)
        inv_lp_out = np.log(1+np.abs(self.fourier_inverse(img_f * lp_filter)))
        
        perwit_x_kernal = np.array([[-1, -1, -1], [0, 0, 0], [1, 1, 1]])
        hp_filter = np.zeros(self.image.shape)
        hp_filter[20:23,20:23] = perwit_x_kernal
        hp_filter = self.fourier(hp_filter)
        inv_hp_out = np.log(1+np.abs(self.fourier_inverse(img_f * hp_filter)))
        
        
        
        self.ui.widget.show()
        self.ui.widget.setImage(np.log(1+np.abs(img_f)))
        self.ui.widget_2.show()
        self.ui.widget_2.setImage(np.log(1+np.abs(lp_filter)))
        self.ui.widget_5.show()
        self.ui.widget_5.setImage(np.log(1+np.abs(hp_filter)))
        self.ui.widget_3.show()
        self.ui.widget_3.setImage(inv_lp_out)
        self.ui.widget_4.show()
        self.ui.widget_4.setImage((inv_hp_out))
        
    def RGB_histo(self):
        self.img_name = self.ui.selectimg.currentText()
        if(self.img_name == "Image 1"):
            path = self.path1
        elif(self.img_name == "Image 2"):
            path = self.path2
        self.image, self.image_gray = self.read_image(path)
        histogram_r, bin_counts = np.histogram(self.image[:,:,0], bins=256)
        histogram_g, bin_counts = np.histogram(self.image[:,:,1], bins=256)
        histogram_b, bin_counts = np.histogram(self.image[:,:,2], bins=256)
        pen1 = pg.mkPen(color=(200, 0, 0))
        pen2 = pg.mkPen(color=(0, 200, 0))
        pen3 = pg.mkPen(color=(0, 0, 200))
        self.histo_r.setData(histogram_r,pen=pen1)
        self.histo_g.setData(histogram_g,pen=pen2)
        self.histo_b.setData(histogram_b,pen=pen3)
    
    
    def low_pass(self,image):
        gaussiankernel = self.gkern(5, 21)
        lowpassed = self.conv(image, gaussiankernel)
        return lowpassed

    def high_pass(self,image):
        laplaciankernel = np.array([[0, -1, 0], [-1, 4, -1], [0, -1, 0]])
        highpassed = self.conv(image, laplaciankernel)
        return highpassed

    def hybridImage(self):
        self.img_name = self.ui.selectimg.currentText()
        path_1 = self.path1
        path_2 = self.path2
        self.image_color, self.image1 = self.read_image(path_1)
        self.image_color, self.image2 = self.read_image(path_2)
        highPassed = self.high_pass(self.image1)
        lowPassed = self.low_pass(self.image2)
        hybrid = highPassed + lowPassed
        
        self.ui.hybrid_view1.show()
        self.ui.hybrid_view1.setImage(highPassed)
        self.ui.hybrid_view2.show()
        self.ui.hybrid_view2.setImage(lowPassed)
        self.ui.hybrid_view3.show()
        self.ui.hybrid_view3.setImage(hybrid)
        
        
        
    
        
        
        
        
    def error(self,value):
        msg = QMessageBox()
        msg.setWindowTitle("error message")
        msg.setText(value)  
        msg.setIcon(QMessageBox.Critical)
        x=msg.exec_()
    

def main():
    app = QtWidgets.QApplication(sys.argv)
    application = ApplicationWindow()
    application.show()
    app.exec_()


if __name__ == "__main__":
    main()

RuntimeError: wrapped C/C++ object of type QComboBox has been deleted

RuntimeError: wrapped C/C++ object of type QComboBox has been deleted

RuntimeError: wrapped C/C++ object of type QComboBox has been deleted

RuntimeError: wrapped C/C++ object of type QComboBox has been deleted

RuntimeError: wrapped C/C++ object of type QComboBox has been deleted

...

RuntimeError: wrapped C/C++ object of type QComboBox has been deleted

RuntimeError: wrapped C/C++ object of type QComboBox has been deleted

RuntimeError: wrapped C/C++ object of type QComboBox has been deleted

RuntimeError: wrapped C/C++ object of type QComboBox has been deleted

RuntimeError: wrapped C/C++ object of type QComboBox has been deleted